In [3]:
from imblearn.combine import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *

In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk
import datetime as dt

# import matplotlib as mpl
# mpl.use('Agg')
# import matplotlib.pylab as plt
# from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from sklearn.preprocessing import Imputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()
''
import warnings

warnings.filterwarnings('ignore')

In [5]:
# train 
detail_train = pd.read_csv('./coupon_data_project2/coupon_detail_train_translated_en.csv', parse_dates=['I_DATE'])
visit_train = pd.read_csv('./coupon_data_project2/coupon_visit_train.csv', parse_dates=['I_DATE'])

area_train = pd.read_csv('./coupon_data_project2/coupon_area_train_translated_en.csv')
coupon_list_train = pd.read_csv('./coupon_data_project2/coupon_list_train_translated_en.csv', parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])

# base data
location = pd.read_csv('./coupon_data_project2/train_location.csv')
user_list = pd.read_csv('./coupon_data_project2/user_list_translated_en.csv', parse_dates=['WITHDRAW_DATE', 'REG_DATE'])

# test data
area_test = pd.read_csv('./coupon_data_project2/test_location.csv')
coupon_list_test = pd.read_csv('./coupon_data_project2/coupon_list_test_translated_en.csv', parse_dates=['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND'])

# submisiion
submission_sample = pd.read_csv('./coupon_data_project2/sample_submission.csv')

In [6]:
# 테이블 현황 파악을 위한 함수 
def summary_table(table):
    df = pd.DataFrame()
    for i in table.columns:
        name = i
        dtype = table[i].dtype.name
        null = table[i].isnull().sum()
        act = table.shape[0] - null
        unique = len(table[i].unique())
        data = {'name': name, 'dtype': dtype, 'null': null, 'act': act, 'unique': unique}
        df = df.append(data, ignore_index=True)
    return df

In [31]:
detail_train

,USER_ID_hash,COUPON_ID_hash
0,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
5,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
6,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
7,560574a339f1b25e57b0221e486907ed,259a22533cab381464c89f616a56d12d
8,560574a339f1b25e57b0221e486907ed,f4a6f861a266799b07cd99c26e80962b
9,560574a339f1b25e57b0221e486907ed,4257a8a169b40bbbcb002ec47ab3de13


In [7]:
detail_train.rename(columns = {'I_DATE': 'purchase_date'}, inplace=True)
detail_train.rename(columns = {'SMALL_AREA_NAME': 'resid_small'}, inplace=True)

In [8]:
detail_train.drop(labels = ['ITEM_COUNT'], axis=1, inplace=True)
detail_train.drop(labels = ['PURCHASEID_hash'], axis=1, inplace=True)
detail_train.drop(labels = ['resid_small'], axis=1, inplace=True)
detail_train.drop(labels = ['purchase_date'], axis=1, inplace=True)

In [9]:
coupon_list = pd.merge(coupon_list_test, coupon_list_train, how='outer')

In [10]:
coupon_list_test.shape, coupon_list_train.shape, coupon_list.shape

((310, 24), (19413, 24), (19723, 24))

In [11]:
# 캡슐과 장르 통합 및 명칭 변경
coupon_list['Case'] = coupon_list['CAPSULE_TEXT'] + coupon_list['GENRE_NAME']
coupon_list['Case'] = coupon_list['Case'].apply(lambda x: "HOTEL" if x == 'Guest houseHotel and Japanese hotel' 
                          or x == 'HotelHotel and Japanese hotel'
                          or x == 'Japanese hotelHotel and Japanese hotel'
                          or x == 'Japanse guest houseHotel and Japanese hotel'
                          or x == 'LodgeHotel and Japanese hotel'
                          or x == 'Public hotelHotel and Japanese hotel'
                          or x == 'Resort innHotel and Japanese hotel'
                          or x == 'Vacation rentalHotel and Japanese hotel'
                          else "NAIL" if x == 'Nail and eye salonNail and eye salon'
                          else "HAIR" if x == 'Hair salonHair salon'
                          else "FOOD" if x == 'FoodFood'
                          else "SPA" if x == 'SpaSpa'
                          else "BEAUTY" if x == 'BeautyBeauty'
                          else "CLASS" if x == 'ClassLesson'
                          else "CORRESPONDENCE" if x == 'Correspondence courseLessonClassLesson'
                          else "DELIVERY" if x == 'Delivery serviceDelivery service'
                          else "EVENT" if x == 'EventOther coupon'
                          else "GIFT" if x == 'Gift cardGift card'
                          else "HEALTH" if x == 'Health and medicalHealth and medical'
                          else "LEISURE" if x == 'LeisureLeisure'
                          else "LESSON" if x == 'LessonLesson'
                          else "OTHER" if x == 'OtherOther coupon'
                          else "RELAXATION" if x == 'RelaxationRelaxation'
                          else "WEB" if x == 'Web serviceOther coupon'
                          else 'OTHER'
                          )

In [12]:
# 실판매가 게산
coupon_list['Price'] = coupon_list['CATALOG_PRICE'] - coupon_list['DISCOUNT_PRICE']

In [13]:
# 실판매가 정규화
coupon_list["lnDPRICE"] = np.log1p(coupon_list["Price"])
coupon_list["mDPRICE"] = coupon_list.groupby("Case")["lnDPRICE"].transform(np.mean)
coupon_list["sDPRICE"] = coupon_list.groupby("Case")["lnDPRICE"].transform(np.std)
coupon_list["zprice"] = (coupon_list["lnDPRICE"] - coupon_list["mDPRICE"]) / coupon_list["sDPRICE"]

In [14]:
# 지역명 변경
coupon_list.rename(columns = {"LARGE_AREA_NAME": "spot_large", 
                              "ken_name": "spot_pref", 
                              "SMALL_AREA_NAME": "spot_small"}, inplace=True)

In [17]:
coupon_list.drop(del_ls,1, inplace=True)

In [ ]:
# usable: nan -> 1, 2 -> 0
coupon_list['USABLE_DATE_MON'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_TUE'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_WED'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_THU'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_FRI'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_SAT'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_SUN'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_HOLIDAY'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)
coupon_list['USABLE_DATE_BEFORE_HOLIDAY'].replace([0,2,1,np.nan],[0,0,1,1], inplace=True)

In [23]:
coupon_list.columns

Index(['PRICE_RATE', 'DISPPERIOD', 'VALIDPERIOD', 'COUPON_ID_hash', 'Case',
       'zprice'],
      dtype='object')

In [88]:
coupon_list.isnull().sum()

PRICE_RATE           0
DISPPERIOD           0
VALIDPERIOD       6243
COUPON_ID_hash       0
Case                 0
zprice               0
dtype: int64

In [89]:
coupon_list['VALIDPERIOD'] = coupon_list['VALIDPERIOD'].fillna(179)

In [22]:
coupon_list.drop(labels = ['CAPSULE_TEXT'], axis=1, inplace=True )
coupon_list.drop(labels = ['sDPRICE'], axis=1, inplace=True )


In [20]:
# coupon_list.drop(labels = ['CAPSULE_TEXT'], axis=1, inplace=True )
coupon_list.drop(labels = ['GENRE_NAME'], axis=1, inplace=True )
coupon_list.drop(labels = ['CATALOG_PRICE'], axis=1, inplace=True )
coupon_list.drop(labels = ['DISCOUNT_PRICE'], axis=1, inplace=True )
coupon_list.drop(labels = ['DISPFROM'], axis=1, inplace=True )
coupon_list.drop(labels = ['DISPEND'], axis=1, inplace=True )
coupon_list.drop(labels = ['VALIDFROM'], axis=1, inplace=True )
coupon_list.drop(labels = ['VALIDEND'], axis=1, inplace=True )
coupon_list.drop(labels = ['lnDPRICE'], axis=1, inplace=True )
coupon_list.drop(labels = ['mDPRICE'], axis=1, inplace=True )
coupon_list.drop(labels = ['Price'], axis=1, inplace=True )
coupon_list.drop(labels = ['spot_pref'], axis=1, inplace=True )  # 판단이슈 
coupon_list.drop(labels = ['spot_small'], axis=1, inplace=True ) # 판단이슈 
coupon_list.drop(labels = ['spot_large'], axis=1, inplace=True ) # 판단이슈 
# coupon_list.drop(labels = ['large_area_name'], axis=1, inplace=True ) # 판단이슈 

ValueError: labels ['GENRE_NAME'] not contained in axis

In [ ]:
coupon_list.columns

In [101]:
ls_dummy = ['Case']

In [102]:
coupon_list = pd.get_dummies(coupon_list, columns = ls_dummy)

In [90]:
# train set과 test set을 다시 분리
coupon_list_train = coupon_list[311:]
coupon_list_test = coupon_list[:310]

# detail

In [94]:
detail_train.head()

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
0,1,2012-03-28 15:06:00,Hyogo,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,1,2011-07-04 23:52:00,"GinzaShinbashi,Tokyo,Ueno",1b4eb2435421ede98c8931c42e8220ec,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,1,2011-07-16 00:52:00,"Ebisu,MeguroShinagawa",36b5f9ba46c44b65587d0b16f2e4c77f,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,1,2011-07-16 00:54:00,"Ebisu,MeguroShinagawa",2f30f46937cc9004774e576914b2aa1a,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,1,2011-07-16 00:55:00,"Ebisu,MeguroShinagawa",4d000c64a55ac573d0ae1a8f03677f50,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2


In [95]:
detail_train = pd.merge(detail_train, coupon_list, how='left', on='COUPON_ID_hash')



In [182]:
# 2. replace hash string ---------------------------------
user_ids = user_list.USER_ID_hash
coupon_train_ids = coupon_list_train.COUPON_ID_hash

# for test coupon, added 1000000 to IDs
coupon_test_ids = coupon_list_test.COUPON_ID_hash
coupon_test_ids.rename(lambda x : x + 1000000, inplace=True)
coupon_ids = pd.concat([coupon_train_ids, coupon_test_ids])

def replace_hash(hashes, hash_id_table):
    replace_table = pd.Series(hash_id_table.index, index=hash_id_table.values)
    return replace_table[hashes].values

# replace hash and drop extra columns
detail = detail_train.copy()
detail["USER_ID"] = replace_hash(detail.USER_ID_hash, user_ids)
detail["COUPON_ID"] = replace_hash(detail.COUPON_ID_hash, coupon_ids)
detail = detail[["USER_ID", "COUPON_ID"]]

In [97]:
detail

,USER_ID,COUPON_ID
0,0,15727.0
1,1,838.0
2,1,1222.0
3,1,1222.0
4,1,1222.0
5,1,1222.0
6,1,1222.0
7,1,4080.0
8,1,6696.0
9,1,8787.0


In [98]:
len(np.unique(coupon_list_train.COUPON_ID_hash.values))

19412

In [100]:
detail_train.columns

Index(['ITEM_COUNT', 'I_DATE', 'SMALL_AREA_NAME', 'PURCHASEID_hash',
       'USER_ID_hash', 'COUPON_ID_hash', 'PRICE_RATE', 'DISPPERIOD',
       'VALIDPERIOD', 'Case', 'zprice', 'SEX_ID', 'AGE', 'A'],
      dtype='object')

In [103]:
# test 데이터 생성
coupon_list_test['A'] = 1
user_list['A'] = 1
test = pd.merge(coupon_list_test, user_list, how='outer')

In [107]:
test = pd.get_dummies(test, columns = ls_dummy)
test.drop(labels = ['A'], axis=1, inplace=True)

In [111]:
compare_not_test = [i for i in detail_train.columns if i not in test.columns]
compare_not_train = [i for i in test.columns if i not in detail_train.columns]
print('only_train: {}  \n'.format(compare_not_test))
print('only_test: {}'.format(compare_not_train))

only_train: ['Case_BEAUTY', 'Case_EVENT', 'Case_LESSON']  

only_test: []


### test 생성

In [58]:
test = coupon_list.filter(detail_train.columns)

In [114]:
train = detail_train.copy()

### B2. train data set 
이후부터 

In [115]:
numerical_features = train.select_dtypes(exclude = ["object"]).columns
categorical_features = train.select_dtypes(include = ["object"]).columns

In [116]:
continuous = list(set(numerical_features) - set(categorical_features))

In [117]:
categorical_features

Index(['USER_ID_hash', 'COUPON_ID_hash'], dtype='object')

In [118]:
test.columns

Index(['PRICE_RATE', 'DISPPERIOD', 'VALIDPERIOD', 'COUPON_ID_hash', 'zprice',
       'SEX_ID', 'AGE', 'USER_ID_hash', 'Case_CLASS', 'Case_DELIVERY',
       'Case_FOOD', 'Case_GIFT', 'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL',
       'Case_LEISURE', 'Case_NAIL', 'Case_OTHER', 'Case_RELAXATION',
       'Case_SPA', 'Case_WEB', 'Case_BEAUTY', 'Case_EVENT', 'Case_LESSON'],
      dtype='object')

In [119]:
test2 = test.copy()

In [120]:
test2.drop('USER_ID_hash', inplace=True, axis=1)

In [121]:
test_features = test2

In [125]:
test_features.shape

(7090630, 23)

In [146]:
train_drop_c = train.drop('COUPON_ID_hash', axis=1)
user = train_drop_c.groupby(by='USER_ID_hash').apply(np.mean)

KeyboardInterrupt: 

In [149]:
del_ls = ['COUPON_ID_hash','SEX_ID', 'AGE']
train_drop_c = train.drop(del_ls, axis=1)

In [166]:
user.columns

Index(['PRICE_RATE', 'DISPPERIOD', 'VALIDPERIOD', 'zprice', 'Case_BEAUTY',
       'Case_CLASS', 'Case_DELIVERY', 'Case_EVENT', 'Case_FOOD', 'Case_GIFT',
       'Case_HAIR', 'Case_HEALTH', 'Case_HOTEL', 'Case_LEISURE', 'Case_LESSON',
       'Case_NAIL', 'Case_OTHER', 'Case_RELAXATION', 'Case_SPA', 'Case_WEB'],
      dtype='object')

In [161]:
user.drop(['SEX_ID', 'AGE'],1, inplace=True)

In [164]:
coupon_list_test.columns

Index(['PRICE_RATE', 'DISPPERIOD', 'VALIDPERIOD', 'COUPON_ID_hash', 'zprice',
       'Case_CLASS', 'Case_DELIVERY', 'Case_FOOD', 'Case_GIFT', 'Case_HAIR',
       'Case_HEALTH', 'Case_HOTEL', 'Case_LEISURE', 'Case_NAIL', 'Case_OTHER',
       'Case_RELAXATION', 'Case_SPA', 'Case_WEB', 'Case_BEAUTY', 'Case_EVENT',
       'Case_LESSON'],
      dtype='object')

In [156]:
coupon_list_test = pd.get_dummies(coupon_list_test, columns = ls_dummy)
coupon_list_test.drop(labels = ['A'], axis=1, inplace=True)

In [159]:
coupon_list_test['Case_BEAUTY'] = 0
coupon_list_test['Case_EVENT'] = 0 
coupon_list_test['Case_LESSON'] = 0 

In [163]:
compare_not_test = [i for i in user.columns if i not in coupon_list_test.columns]
compare_not_train = [i for i in coupon_list_test.columns if i not in user.columns]
print('only_train: {}  \n'.format(compare_not_test))
print('only_test: {}'.format(compare_not_train))

only_train: []  

only_test: ['COUPON_ID_hash']


In [130]:
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale


In [131]:
from sklearn.preprocessing import *


In [ ]:
imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

In [167]:
test_features = coupon_list_test
test_features = test_features.drop('COUPON_ID_hash', axis=1)
test_features.columns

Index(['PRICE_RATE', 'DISPPERIOD', 'VALIDPERIOD', 'zprice', 'Case_CLASS',
       'Case_DELIVERY', 'Case_FOOD', 'Case_GIFT', 'Case_HAIR', 'Case_HEALTH',
       'Case_HOTEL', 'Case_LEISURE', 'Case_NAIL', 'Case_OTHER',
       'Case_RELAXATION', 'Case_SPA', 'Case_WEB', 'Case_BEAUTY', 'Case_EVENT',
       'Case_LESSON'],
      dtype='object')

In [168]:
user.shape, test_features.shape

((22782, 20), (310, 20))

In [169]:
user.isnull().sum() , \
test_features.isnull().sum()

(PRICE_RATE         0
 DISPPERIOD         0
 VALIDPERIOD        0
 zprice             0
 Case_BEAUTY        0
 Case_CLASS         0
 Case_DELIVERY      0
 Case_EVENT         0
 Case_FOOD          0
 Case_GIFT          0
 Case_HAIR          0
 Case_HEALTH        0
 Case_HOTEL         0
 Case_LEISURE       0
 Case_LESSON        0
 Case_NAIL          0
 Case_OTHER         0
 Case_RELAXATION    0
 Case_SPA           0
 Case_WEB           0
 dtype: int64, PRICE_RATE         0
 DISPPERIOD         0
 VALIDPERIOD        0
 zprice             0
 Case_CLASS         0
 Case_DELIVERY      0
 Case_FOOD          0
 Case_GIFT          0
 Case_HAIR          0
 Case_HEALTH        0
 Case_HOTEL         0
 Case_LEISURE       0
 Case_NAIL          0
 Case_OTHER         0
 Case_RELAXATION    0
 Case_SPA           0
 Case_WEB           0
 Case_BEAUTY        0
 Case_EVENT         0
 Case_LESSON        0
 dtype: int64)

In [ ]:
# vector_1.drop(labels = ['distance'], axis=1, inplace=True)
# vector_1.drop(labels = ['VALIDPERIOD'], axis=1, inplace=True)
# vector_2.drop(labels = ['distance'], axis=1, inplace=True)
# vector_2.drop(labels = ['VALIDPERIOD'], axis=1, inplace=True)
vector_1.drop(labels=['Case_EVENT', 'Case_BEAUTY', 'Case_LESSON'],axis=1, inplace=True )

In [170]:
vector_1 = user.copy()
vector_2 = test_features.copy()

vector_1 = scale(vector_1)
vector_1 = pd.DataFrame(data = vector_1, index = user.index, columns = user.columns )

vector_2 = scale(vector_2)
vector_2 = pd.DataFrame(data = vector_2, index = test_features.index, columns = test_features.columns )

In [136]:
import math

def Cosine(vec1, vec2) :
    result = InnerProduct(vec1,vec2) / (VectorSize(vec1) * VectorSize(vec2))
    return result

def VectorSize(vec) :
    return math.sqrt(sum(math.pow(v,2) for v in vec))

def InnerProduct(vec1, vec2) :
    return sum(v1*v2 for v1,v2 in zip(vec1,vec2))

def Euclidean(vec1, vec2) :
    return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(vec1, vec2)))

def Theta(vec1, vec2) :
    return math.acos(Cosine(vec1,vec2)) + 10

def Triangle(vec1, vec2) :
    theta = math.radians(Theta(vec1,vec2))
    return (VectorSize(vec1) * VectorSize(vec2) * math.sin(theta)) / 2

def Magnitude_Difference(vec1, vec2) :
    return abs(VectorSize(vec1) - VectorSize(vec2))

def Sector(vec1, vec2) :
    ED = Euclidean(vec1, vec2)
    MD = Magnitude_Difference(vec1, vec2)
    theta = Theta(vec1, vec2)
    return math.pi * math.pow((ED+MD),2) * theta/360

def TS_SS(vec1, vec2) :
    return Triangle(vec1, vec2) * Sector(vec1, vec2)

vec1 = [1,2]
vec2 = [2,4]

print(Euclidean(vec1,vec2))
print(Cosine(vec1,vec2))
print(TS_SS(vec1,vec2))

2.23606797749979
0.9999999999999998
1.5153662265423102


In [173]:
vector_2.shape

(310, 20)

In [174]:
from scipy import spatial
def cosine_similarity(vector_1, vector_2):

    return 1 - spatial.distance.cosine(vector_1, vector_2)

In [171]:
!pip install tqdm

In [180]:
from tqdm import tqdm

In [181]:
matrix = []
for idx1 in tqdm(range(len(user))):
    
    row = []
    for idx2 in range(len(test_features)):
        row.append(cosine_similarity(vector_1.iloc[idx1], vector_2.iloc[idx2]))
    matrix.append(row)

100%|██████████| 22782/22782 [41:42<00:00,  9.11it/s]


In [196]:
test_features

,PRICE_RATE,DISPPERIOD,VALIDPERIOD,zprice,Case_CLASS,Case_DELIVERY,Case_FOOD,Case_GIFT,Case_HAIR,Case_HEALTH,Case_HOTEL,Case_LEISURE,Case_NAIL,Case_OTHER,Case_RELAXATION,Case_SPA,Case_WEB,Case_BEAUTY,Case_EVENT,Case_LESSON
0,52,4,118.0,0.458796,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,52,7,100.0,2.353907,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,51,7,175.0,0.826188,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,50,5,123.0,-0.498151,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,56,7,129.0,-0.144587,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,50,7,156.0,1.684594,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,51,7,151.0,1.555460,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,50,7,87.0,0.691574,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,57,7,115.0,-1.380101,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,62,7,116.0,1.314516,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
continuous = list(set(vector_1) - set(vector_2))
continuous

In [183]:
coupons_ids = test['COUPON_ID_hash']
result_index = vector_1.index
result_columns = [coupons_ids[i] for i in range(0, 310)]
result_df = pd.DataFrame(index= result_index, columns=result_columns,
                      data=matrix)

In [ ]:
submission2 = pd.DataFrame(columns=['USER_ID_hash','PURCHASED_COUPONS'])
submission2['USER_ID_hash'] = result_df.index

In [250]:
submission2['PURCHASED_COUPONS'] = result_df.T.apply(lambda x: " ".join(x.sort_values(ascending=False)[:21].index)).values

In [261]:
submission_sample['PURCHASED_COUPONS'] = submission_sample['PURCHASED_COUPONS'].fillna(method='backfill')

In [207]:
submission.to_csv('cosine.csv',index = False,header=True)


In [197]:
coupons_ids = coupon_list_test['COUPON_ID_hash']

In [201]:
len(coupons_ids)#, len()

310

In [203]:
#### create (USED_ID)x(COUPON_ID) dataframe, similarity scores as values
# coupons_ids = coupon_list_test['COUPON_ID_hash']
index = user.index
columns = coupons_ids#[coupons_ids[i] for i in range(310)]
result_df = pd.DataFrame(index=index, columns=columns,
                      data=matrix)

In [204]:
result_df

COUPON_ID_hash,c76ea297ebd3a5a4d3bf9f75269f66fa,dd74dc95ca294afa02db40a543ae1763,c65b550cbef918796ad53b1d5b7165c1,f93dc6e223935d817e1237f8f73b56a2,86c64391318f1d751647bf8e1882cf1d,78aec0e250021cf5247986f6ca55a47e,44000991cad71e5105d3ad3b78d92431,3810431a7769cfcc3201383b5e83248c,520cb8509c6662059d3f71e7c3cad0ae,db25ee0094a086ed6663f94e41428835,...,9e5f11a475b172a6384dec160865fcc2,6adfd771501bf0595c0538f6eb60a84a,0c015306597566b632bebfb63b7e59f3,9cfcf41925ff2b183ff55789462333a3,c988d799bc7db9254fe865ee6cf2d4ff,65500c6708fa82d36b3a467a94dd2c50,e3e9027e1b87d957b1ccdb24d128e455,09ec991661af3ac7bc3b5cddc31e2fa0,73675e451067962afc48257474308cd0,f9c657ce7ca80b3766ced3a9a3c709bb
USER_ID_hash,,,,,,,,,,,,,,,,,,,,,
0000b53e182165208887ba65c079fc21,-0.211989,-0.049716,-0.114432,-0.382752,-0.355738,-0.035961,-0.062867,-0.287758,-0.493454,-0.175080,...,-0.138121,-0.309519,-0.086780,0.023190,-0.197172,-0.429269,-0.429269,-0.429269,-0.244022,-0.091651
00035b86e6884589ec8d28fbf2fe7757,-0.195261,-0.218464,-0.131824,-0.176502,-0.248176,-0.147058,-0.164571,-0.240562,-0.238519,-0.312620,...,-0.098547,-0.197220,-0.054461,-0.117179,-0.271268,-0.137233,-0.137233,-0.137233,-0.089446,-0.055046
0005b1068d5f2b8f2a7c978fcfe1ca06,-0.345767,-0.080032,-0.052230,-0.326840,-0.115318,-0.053519,-0.057784,-0.180628,-0.153170,-0.025466,...,-0.056116,-0.159178,0.086247,-0.141210,0.031244,-0.216777,-0.216777,-0.216777,-0.242594,-0.265473
000cc06982785a19e2a2fdb40b1c9d59,-0.060102,-0.015501,-0.098256,-0.052326,-0.014656,-0.082231,-0.073548,0.006138,0.018515,0.011142,...,0.027563,-0.035857,-0.001694,0.429469,-0.087681,-0.053939,-0.053939,-0.053939,-0.274021,0.393923
0013518e41c416cd6a181d277dd8ca0b,-0.354353,0.194641,0.018622,-0.237617,0.127411,0.072940,0.087652,0.184074,0.110368,0.237053,...,0.140642,0.096583,0.201054,0.053140,0.543482,-0.134607,-0.134607,-0.134607,0.195391,0.005069
001acdee812a18acfd7509172bed5700,0.467719,0.353002,0.603102,0.474361,0.507233,0.525636,0.525809,0.342773,0.398666,0.472999,...,-0.171140,0.470183,0.038234,0.026361,-0.035561,0.489285,0.489285,0.489285,-0.429618,-0.065700
001fd7876e3aa29393537c6baf308e43,0.665385,0.327309,0.477582,0.473522,0.305575,0.458440,0.448420,0.220597,0.149162,0.363057,...,-0.185335,0.301460,-0.122340,-0.132854,-0.247194,0.327798,0.327798,0.327798,-0.300778,-0.234875
002383753c1e5d6305c8aff6f89e26d6,-0.234172,-0.286369,-0.192867,-0.190010,-0.256606,-0.224320,-0.236430,-0.274786,-0.206955,-0.341743,...,-0.072372,-0.228639,-0.040214,-0.141168,-0.209906,-0.128458,-0.128458,-0.128458,-0.081137,-0.080090
0025cae7997d25ea5cf8851bb099c798,-0.376036,-0.071351,-0.111562,-0.303068,-0.152768,-0.086845,-0.093852,-0.106162,-0.159649,-0.132649,...,-0.018861,-0.125554,0.059204,0.234272,-0.140969,-0.216533,-0.216533,-0.216533,-0.165070,0.233355


In [205]:
### obtain string of top10 hashes according to similarity scores for every user
def get_top10_coupon_hashes_string(row):
    row.sort()
    return ' '.join(row.index[-10:][::-1].tolist())

output = result_df.apply(get_top10_coupon_hashes_string, axis=1)


output_df = pd.DataFrame(data={'USER_ID_hash': output.index,
                               'PURCHASED_COUPONS': output.values})
output_df_all_users = pd.merge(user_list, output_df, how='left', on='USER_ID_hash')
output_df_all_users.to_csv('cosine_sim_python.csv', header=True,
                           index=False, columns=['USER_ID_hash', 'PURCHASED_COUPONS'])

AttributeError: ("'Series' object has no attribute 'sort'", 'occurred at index 0000b53e182165208887ba65c079fc21')

In [209]:
submission_sample = pd.read_csv('./coupon_data_project2/sample_submission.csv')

In [223]:
compare_not_test = [i for i in list(user_list['USER_ID_hash']) if i not in list(submission['USER_ID_hash'])]
# compare_not_train = [i for i in submission['USER_ID_hash'] if i not in submission_sample['USER_ID_hash']]
print('only_train: {}  \n'.format(compare_not_test))
# print('only_test: {}'.format(compare_not_train))

only_train: ['f243e9cafe5464014facff10eb36ae8e', 'f03041e2114dcff47afe17a4a6b1c95b', '63aa41b24b76cdde8b3e756c3c107a6a', 'a430df69523d291460db24b076297d81', 'e77d22a4fa0603aaca0f7774db8d04d2', '8e0d7f4cdb2e4550c68a0dd4cb85f45e', '4148480f120ebc38eb760b66c2c0bb74', 'd205f3a89819f0bc74cc0d15070864ab', '536e7e4555d6f484d2452a8c334fe740', 'c068d179b8e1f82205949d0c7d91c7ca', 'ffb0df1012b40db425574eba2541f546', '9a99789e321ca426c6554891dcf87310', '6168a7883b13479db3d87a7e991d5735', '739c3173f1aaa8ffd2e8142265c58f36', '1fe74103ba1d127445af7f4905173141', '78b573de516c03adf91014c0663b1d36', '51e226cb7ac86742d1e884fa108e298b', 'a4a307c9709ebaee717ea086779cf910', '1e0eff90a552136820a20ce07fcf5918', '71dea58765d736bef41ac6557a6318c5', '8c9e35e7a6e06fdc71b30e53d3a26180', '6114083e0a0045d135aa80e000ae4afa', '5fc9c05ec3a038d56faf36baa7d33acf', 'ac35088490db0fb1a230d95c50ae6cb1', '3be924043f64ca2503ed0f1af8ba5558', '66e661b3b2a4a3ba961bb3c1613b2bcb', 'e14aeda5f2629698dc2003392191374b', 'd6258f6d0cf841

In [225]:
len(compare_not_test)

91

In [ ]:
submission_sample.insert(2, 'distance', train['key'].map(location.set_index('key')['distance']))


In [233]:
submission_sample.insert(1, 'PURCHASED_COUPONS', submission_sample['USER_ID_hash'].map(submission.set_index('USER_ID_hash')['PURCHASED_COUPONS']))

In [228]:
submission_sample.drop('PURCHASED_COUPONS',1, inplace=True)

In [262]:
submission_sample.to_csv('test.csv',index=False)

In [238]:
submission_sample.isnull().sum()

USER_ID_hash          0
PURCHASED_COUPONS    91
dtype: int64

In [ ]:
# 2. replace hash string ---------------------------------
user_ids = user_list.USER_ID_hash
coupon_train_ids = coupon_list_train.COUPON_ID_hash

# for test coupon, added 1000000 to IDs
coupon_test_ids = coupon_list_test.COUPON_ID_hash
coupon_test_ids.rename(lambda x : x + 1000000, inplace=True)
coupon_ids = pd.concat([coupon_train_ids, coupon_test_ids])

def replace_hash(hashes, hash_id_table):
    replace_table = pd.Series(hash_id_table.index, index=hash_id_table.values)
    return replace_table[hashes].values

# replace hash and drop extra columns
detail = detail_train.copy()
detail["USER_ID"] = replace_hash(detail.USER_ID_hash, user_ids)
detail["COUPON_ID"] = replace_hash(detail.COUPON_ID_hash, coupon_ids)
detail = detail[["USER_ID", "COUPON_ID"]]

In [240]:
### create (USED_ID)x(COUPON_ID) dataframe, similarity scores as values
coupons_ids = coupon_list_test['COUPON_ID_hash']

In [243]:
coupons_ids = coupon_list_test['COUPON_ID_hash']
index = user.index
columns = list(coupons_ids)
result_df = pd.DataFrame(index=index, columns=columns,
                      data=matrix)

In [248]:
result_df.shape

(22782, 310)

In [247]:
def get_top10_coupon_hashes_string(row):
    row.sort()
    return ' '.join(row.index[-10:][::-1].tolist())

output = result_df.apply(get_top10_coupon_hashes_string, axis=1)

AttributeError: ("'Series' object has no attribute 'sort'", 'occurred at index 0000b53e182165208887ba65c079fc21')

In [ ]:
coupons_ids = coupon_list_test['COUPON_ID_hash']
index = user.index
columns = list(coupons_ids)
result_df = pd.DataFrame(index=index, columns=columns,
                      data=matrix)

### obtain string of top10 hashes according to similarity scores for every user
def get_top10_coupon_hashes_string(row):
    row.sort()
    return ' '.join(row.index[-10:][::-1].tolist())

output = result_df.apply(get_top10_coupon_hashes_string, axis=1)


output_df = pd.DataFrame(data={'USER_ID_hash': output.index,
                               'PURCHASED_COUPONS': output.values})
output_df_all_users = pd.merge(user_list, output_df, how='left', on='USER_ID_hash')
output_df_all_users.to_csv('cosine_sim_python.csv', header=True,
                           index=False, columns=['USER_ID_hash', 'PURCHASED_COUPONS'])